In [1]:

%matplotlib inline

import pandas as pd
import sklearn
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random
mpl.pyplot.style.use('ggplot')
from graphviz import Digraph

In [2]:
loandata = pd.read_csv('../data/LoanStats3d.csv',skiprows=[0])

C:\Users\New\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
completedloans=loandata[(loandata.loan_status=='Charged Off')|(loandata.loan_status=='Fully Paid')]

In [4]:
# Split into test and training sets. Allocating 80% for training data
random.seed(1234)
traindata, testdata = train_test_split(completedloans, test_size = 0.2)

## Data Cleaning


In [5]:
traindata.drop(['id','member_id','funded_amnt','emp_title','desc','url','pymnt_plan','title'],1,inplace=True)
#traindata.drop('id',1,inplace=True)


C:\Users\New\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
from datetime import datetime
traindata.earliest_cr_line = pd.to_datetime(traindata.earliest_cr_line)
traindata.last_pymnt_d = pd.to_datetime(traindata.last_pymnt_d)
traindata.next_pymnt_d = pd.to_datetime(traindata.next_pymnt_d)
traindata.last_credit_pull_d = pd.to_datetime(traindata.last_credit_pull_d)

C:\Users\New\Anaconda2\lib\site-packages\pandas\core\generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [7]:
traindata.earliest_cr_line =[datetime.toordinal(x) for x in traindata.earliest_cr_line]
traindata.last_pymnt_d =[datetime.toordinal(x) for x in traindata.last_pymnt_d]
traindata.next_pymnt_d =[datetime.toordinal(x) for x in traindata.next_pymnt_d]
traindata.last_credit_pull_d =[datetime.toordinal(x) for x in traindata.last_credit_pull_d]

In [8]:
traindata.int_rate = pd.Series(traindata.int_rate).str.replace('%', '').astype(float)
traindata.revol_util= pd.Series(traindata.revol_util).str.replace('%', '').astype(float)

In [9]:
testdata.earliest_cr_line = pd.to_datetime(testdata.earliest_cr_line)
testdata.last_pymnt_d = pd.to_datetime(testdata.last_pymnt_d)
testdata.next_pymnt_d = pd.to_datetime(testdata.next_pymnt_d)
testdata.last_credit_pull_d = pd.to_datetime(testdata.last_credit_pull_d)
testdata.earliest_cr_line =[datetime.toordinal(x) for x in testdata.earliest_cr_line]
testdata.last_pymnt_d =[datetime.toordinal(x) for x in testdata.last_pymnt_d]
testdata.next_pymnt_d =[datetime.toordinal(x) for x in testdata.next_pymnt_d]
testdata.last_credit_pull_d =[datetime.toordinal(x) for x in testdata.last_credit_pull_d]
testdata.int_rate = pd.Series(testdata.int_rate).str.replace('%', '').astype(float)
testdata.revol_util= pd.Series(testdata.revol_util).str.replace('%', '').astype(float)

In [10]:
# Import SciKit Learn functions
from sklearn.metrics import roc_curve, auc , roc_auc_score, confusion_matrix, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier , export_graphviz
from sklearn.cross_validation import train_test_split
from sklearn.externals.six import StringIO
from IPython.display import Image
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [11]:
# Rearranging the columns so that the response variable is at index 0
cols = traindata.columns.tolist()
cols =(cols[12:13]+cols[0:12]+cols[13:])
traindata = traindata[cols]

In [12]:
testdata = testdata[cols]

In [13]:
# This code transforms the string columns into numerical values 
le = preprocessing.LabelEncoder()
le.fit(traindata.home_ownership)
le.classes_
traindata.home_ownership=le.transform(traindata.home_ownership)

le.fit(traindata.term)
le.classes_
traindata.term=le.transform(traindata.term)

le.fit(traindata.grade)
le.classes_
traindata.grade=le.transform(traindata.grade)


le.fit(traindata.loan_status)
le.classes_
traindata.loan_status=le.transform(traindata.loan_status)

le.fit(traindata.emp_length)
le.classes_
traindata.emp_length=le.transform(traindata.emp_length)

le.fit(traindata.verification_status)
le.classes_
traindata.verification_status=le.transform(traindata.verification_status)

le.fit(traindata.issue_d)
le.classes_
traindata.issue_d=le.transform(traindata.issue_d)

le.fit(traindata.purpose)
le.classes_
traindata.purpose=le.transform(traindata.purpose)

le.fit(traindata.zip_code)
le.classes_
traindata.zip_code=le.transform(traindata.zip_code)

le.fit(traindata.addr_state)
le.classes_
traindata.addr_state=le.transform(traindata.addr_state)

le.fit(traindata.initial_list_status)
le.classes_
traindata.initial_list_status=le.transform(traindata.initial_list_status)

le.fit(traindata.application_type)
le.classes_
traindata.application_type=le.transform(traindata.application_type)

le.fit(traindata.verification_status_joint)
le.classes_
traindata.verification_status_joint=le.transform(traindata.verification_status_joint)


le.fit(traindata.sub_grade)
le.classes_
traindata.sub_grade=le.transform(traindata.sub_grade)
# Split the target from the input variables
X_train = traindata.iloc[:, 1:]
y_train = traindata.iloc[:, 0]

C:\Users\New\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:200: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  flag = np.concatenate(([True], aux[1:] != aux[:-1]))
C:\Users\New\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:259: FutureWarning: numpy equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  return aux[:-1][aux[1:] == aux[:-1]]


In [14]:
# This code transforms the string columns into numerical values 
le = preprocessing.LabelEncoder()
le.fit(testdata.home_ownership)
le.classes_
testdata.home_ownership=le.transform(testdata.home_ownership)

le.fit(testdata.term)
le.classes_
testdata.term=le.transform(testdata.term)

le.fit(testdata.grade)
le.classes_
testdata.grade=le.transform(testdata.grade)


le.fit(testdata.loan_status)
le.classes_
testdata.loan_status=le.transform(testdata.loan_status)

le.fit(testdata.emp_length)
le.classes_
testdata.emp_length=le.transform(testdata.emp_length)

le.fit(testdata.verification_status)
le.classes_
testdata.verification_status=le.transform(testdata.verification_status)

le.fit(testdata.issue_d)
le.classes_
testdata.issue_d=le.transform(testdata.issue_d)

le.fit(testdata.purpose)
le.classes_
testdata.purpose=le.transform(testdata.purpose)

le.fit(testdata.zip_code)
le.classes_
testdata.zip_code=le.transform(testdata.zip_code)

le.fit(testdata.addr_state)
le.classes_
testdata.addr_state=le.transform(testdata.addr_state)

le.fit(testdata.initial_list_status)
le.classes_
testdata.initial_list_status=le.transform(testdata.initial_list_status)

le.fit(testdata.application_type)
le.classes_
testdata.application_type=le.transform(testdata.application_type)

le.fit(testdata.verification_status_joint)
le.classes_
testdata.verification_status_joint=le.transform(testdata.verification_status_joint)


le.fit(testdata.sub_grade)
le.classes_
testdata.sub_grade=le.transform(testdata.sub_grade)
# Split the target from the input variables
X_test = testdata.iloc[:, 1:]
y_test = testdata.iloc[:, 0]

In [15]:
len(X_train.index)

37216

In [16]:
nullcolls=(X_train.count()!=len(X_train.index))
matches = [i for i in range(0,len(nullcolls)) if i == True]
names=nullcolls[nullcolls]
nadroplist=list(names.index)

In [17]:
X_train.drop(nadroplist,1,inplace=True)
X_test.drop(nadroplist,1,inplace=True)

In [18]:
rfclf = RandomForestClassifier(n_estimators=100, max_features='auto', oob_score=True, random_state=1)


# Fit the random forest classifier
rfclf.fit(X_train, y_train)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [20]:
# compute the out-of-bag classification accuracy
rfclf.oob_score_

0.99946259673258808

In [22]:
probs = rfclf.predict_proba(X_test)[:,1]
metrics.roc_auc_score(y_test,probs)

0.99944732816067317

In [23]:
metrics.accuracy_score(y_test,rfclf.predict(X_test))

0.99924763542562334

In [24]:
pd.crosstab(y_test, rfclf.predict(X_test), rownames=['actual'], colnames=['predicted'])

predicted,0,1
actual,,
0,1525,7
1,0,7772


In [25]:
from sklearn.cross_validation import cross_val_score
cross_val_score(rfclf, X_train, y_train, cv=10, scoring='roc_auc').mean()

0.99986908327036317

In [27]:
# compute the feature importances
importances=pd.DataFrame({'feature':X_train.columns, 'importance':rfclf.feature_importances_})

importances.sort_index(by='importance',ascending=False).head()

C:\Users\New\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,feature,importance
28,total_rec_prncp,0.227802
34,last_pymnt_amnt,0.196463
26,total_pymnt,0.147979
27,total_pymnt_inv,0.120997
31,recoveries,0.053245


Oops, it appears the reason accuray is so high is that some features feed the answer to the learning model. In particular we feed the received principal and total received amounts to date. Ideally we want a model that predicts the final loan status before the loan commences. In this case, we won't have these features available to use as predictors. We'll try to remake the learning models without these features

In [34]:
X_train2=X_train.drop(['total_rec_prncp','last_pymnt_amnt','total_pymnt','total_pymnt_inv','recoveries'],1)
X_test2=X_test.drop(['total_rec_prncp','last_pymnt_amnt','total_pymnt','total_pymnt_inv','recoveries'],1)

In [32]:
# Fit the random forest classifier
rfclf.fit(X_train2, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [33]:
# compute the out-of-bag classification accuracy
rfclf.oob_score_

0.91492906276870167

In [35]:
probs = rfclf.predict_proba(X_test2)[:,1]
metrics.roc_auc_score(y_test,probs)

0.9516539169865984

In [36]:
metrics.accuracy_score(y_test,rfclf.predict(X_test2))

0.91326311263972482

In [37]:
pd.crosstab(y_test, rfclf.predict(X_test2), rownames=['actual'], colnames=['predicted'])

predicted,0,1
actual,,
0,839,693
1,114,7658


In [41]:
cross_val_score(rfclf, X_train2, y_train, cv=10, scoring='roc_auc').mean()

0.94983533371703499

In [40]:
# compute the feature importances
importances=pd.DataFrame({'feature':X_train2.columns, 'importance':rfclf.feature_importances_})

importances.sort_index(by='importance',ascending=False).head()

C:\Users\New\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


,feature,importance
28,collection_recovery_fee,0.191512
29,last_pymnt_d,0.092538
31,last_credit_pull_d,0.087521
26,total_rec_int,0.034413
3,int_rate,0.027325


In [44]:
X_train2.columns

Index([u'loan_amnt', u'funded_amnt_inv', u'term', u'int_rate', u'installment',
       u'grade', u'sub_grade', u'emp_length', u'home_ownership', u'annual_inc',
       u'verification_status', u'issue_d', u'purpose', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv',
       u'total_rec_int', u'total_rec_late_fee', u'collection_recovery_fee',
       u'last_pymnt_d', u'next_pymnt_d', u'last_credit_pull_d',
       u'collections_12_mths_ex_med', u'policy_code', u'application_type',
       u'verification_status_joint', u'acc_now_delinq', u'tot_coll_amt',
       u'tot_cur_bal', u'total_rev_hi_lim', u'acc_open_past_24mths',
       u'avg_cur_bal', u'chargeoff_within_12_mths', u'delinq_amnt',
       u'mo_sin_old_rev_tl_op', u'mo_sin_rcnt_rev_tl_op', u'mo_sin_rcnt_tl',
       u'mort_acc', u'num_accts_ever_120_pd', u'num_actv_bc_tl